In [98]:
import pandas as pd
import numpy as np
import cv2
import torch
import glob as glob
from model import create_model
from config import NUM_CLASSES

In [99]:
# set the computation device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [100]:
# load the model and the trained weights
model = create_model(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(
    '../outputs/model10.pth', map_location=device
))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [103]:
# directory where all the images are present
DIR_TEST = '../test_data'
test_images = glob.glob(f"{DIR_TEST}/*")
print(f"Test instances: {len(test_images)}")
# classes: 0 index is reserved for background
CLASSES = [
    'background', 'button'
]
# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.15

for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split('/')[-1].split('.')[0]
    print(image_name)
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float)
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        outputs = model(image)

    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        # get all the predicited class names
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()]

        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            cv2.rectangle(orig_image,
                          (int(box[0]), int(box[1])),
                          (int(box[2]), int(box[3])),
                          (0, 0, 255), 2)
            cv2.putText(orig_image, pred_classes[j],
                        (int(box[0]), int(box[1] - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0),
                        2, lineType=cv2.LINE_AA)
        cv2.imshow('Prediction', orig_image)
        cv2.waitKey(1)
        cv2.imwrite(f"../test_predictions/{image_name}.jpg", orig_image, )
    print(f"Image {i + 1} done...")
    print('-' * 50)
print('TEST PREDICTIONS COMPLETE')
cv2.destroyAllWindows()

Test instances: 4
image01


/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/3539361090.py:24: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.transpose(image, (2, 0, 1)).astype(np.float)


Image 1 done...
--------------------------------------------------
image03
Image 2 done...
--------------------------------------------------
image02
Image 3 done...
--------------------------------------------------
image04
Image 4 done...
--------------------------------------------------
TEST PREDICTIONS COMPLETE


In [101]:
# directory where all the images are present
DIR_TEST = '../test_data'
test_images = glob.glob(f"{DIR_TEST}/*")
print(f"Test instances: {len(test_images)}")
# classes: 0 index is reserved for background
CLASSES = [
    'background', 'button'
]
# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.16

Test instances: 4


In [ ]:
for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split('/')[-1].split('.')[0]
    print(image_name)
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float64)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float64)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float)
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        results = model(image)

In [89]:
orig_image2 = orig_image.copy()

In [90]:
px = pd.DataFrame(results[0]['boxes'], columns=['x_topleft', 'y_topleft', 'x_bottomright', 'y_bottomright'])
px['labels'] = results[0]['labels']
px['scores'] = results[0]['scores']

detection_threshold = 0.5
show = px.loc[px['scores'] > detection_threshold]
show

,x_topleft,y_topleft,x_bottomright,y_bottomright,labels,scores
0,735.185669,677.049377,772.684753,712.653625,1,0.642759
1,726.318665,558.989319,770.537415,599.043762,1,0.544998


In [91]:
for index, row in show.iterrows():
    cv2.rectangle(orig_image2,
                  (int(row['x_topleft']), int(row['y_topleft'])),
                  (int(row['x_bottomright']), int(row['y_bottomright'])),
                  (0, 255, 0), 1)
cv2.imshow('Prediction', orig_image2)
cv2.waitKey(1)
cv2.imwrite(f"../test_predictions/{image_name}.jpg", orig_image2, )

True

In [104]:
for index, row in show.iterrows():
    show['x_topleft'][index] -= 20
    show['y_topleft'][index] += 50
    show['x_bottomright'][index] += 50
    show['y_bottomright'][index] -= 50
    
    cv2.rectangle(orig_image2,
                  (int(row['x_topleft']), int(row['y_topleft'])),
                  (int(row['x_bottomright']), int(row['y_bottomright'])),
                  (0, 0, 255), 1)
cv2.imshow('Prediction2', orig_image2)
cv2.waitKey(1)
cv2.imwrite(f"../test_predictions/{image_name}.jpg", orig_image2, )

/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/1904810842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show['x_topleft'][index] -= 20
/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/1904810842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show['y_topleft'][index] += 50
/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/1904810842.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

True

In [48]:
cv2.destroyAllWindows()

In [96]:
show['width'] = show['x_bottomright'] - show['x_topleft']
show['height'] = show['y_topleft'] - show['y_bottomright']

/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/2216035249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show['width'] = show['x_bottomright'] - show['x_topleft']
/var/folders/46/wsswctf55vdfp6kz49kfbqtm0000gn/T/ipykernel_3974/2216035249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show['height'] = show['y_topleft'] - show['y_bottomright']


In [97]:
show

,x_topleft,y_topleft,x_bottomright,y_bottomright,labels,scores,width,height
0,695.185669,777.049377,872.684753,612.653625,1,0.642759,177.499084,164.395752
1,686.318665,658.989319,870.537415,499.043762,1,0.544998,184.218750,159.945557


In [ ]:
 print(results)
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in results]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        print(boxes)
        # get all the predicited class names
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()]
    
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            cv2.rectangle(orig_image,
                          (int(box[0]), int(box[1])),
                          (int(box[2]), int(box[3])),
                          (0, 0, 255), 2)
            cv2.putText(orig_image, pred_classes[j],
                        (int(box[0]), int(box[1] - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0),
                        2, lineType=cv2.LINE_AA)
        cv2.imshow('Prediction', orig_image)
        cv2.waitKey(1)
        cv2.imwrite(f"../test_predictions/{image_name}.jpg", orig_image, )
    print(f"Image {i + 1} done...")
    print('-' * 50)
    break
print('TEST PREDICTIONS COMPLETE')
cv2.destroyAllWindows()

In [105]:
cv2.destroyAllWindows()